# Intelligence artificielle - Automne 2025 - Laboratoire 09

## _Prédicats du premier ordre:   Forward Chaining pour les Systèmes basés sur des règles_


## Introduction

Le but de ce laboratoire est de comprendre et de mettre en œuvre la preuve par théorème en utilisant l'enchaînement en avant.

### Clauses définies. Règles. Des faits.

Dans ce laboratoire, nous utiliserons un certain type de formules, plus précisément des **clauses définies**. Ce sont des clauses de Horn avec exactement un littéral positif.

$$\neg p_1 \vee \neg p_2 \vee \ldots \vee \neg p_N \vee c$$

Cela nous amène à:

$$\left( p_1 \wedge p_2 \wedge \ldots \wedge p_N \right) \rightarrow c$$

La forme particulière de ces clauses est similaire aux **règles**.

Nous appellerons **règle** une clause définie avec au moins un littéral négatif et nous appellerons **fait** une clause définie avec un seul littéral (le positif).

### Problème à résoudre

_En donnant une base de connaissances_ `kb` _constituée de clauses définies (faits et règles), on prouve le théorème_ `t`.

## Fonctions utiles des laboratoires précédents

### Tâche 0

Sauvez la solution de `lab 8` (*Représentation et Unification*) avec le nom `Lab08.py`. Nous utiliserons également les fonctions déjà mises en œuvre dans `Devoir3_Resolution`. Les fonctions importantes pour aujourd'hui sont:
  - `make_var`,` make_const`, `make_atom` - utile pour construire des atomes. 
  Par exemple, $$ Weather (tomorrow, Cold) $$ est construit comme suit:
  
`make_atom (" Weather ", make_var (" tomorrow "), make_const (" Cold "))`
 
  - `unify` et `substitute` - utile pour calculer l'unificateur le plus général pour deux formules et pour appliquer une substitution
  - `is_positive_literal` et `is_negative_literal`
  - `add_statement` - ajoute une clause définie à une base de connaissances. Nous avons deux variantes d'utilisation dans ce laboratoire:
    * `add_statement (kb, Atom)` - ajoute le fait à Atom en kb
    * `add_statement (kb, C, P1, P2, Pn)` - ajoute dans kb la règle $\left(p_1 \wedge p_2 \wedge p_N \right) \rightarrow c$ comme:
   
    `make_or (make_neg (P1), make_neg (P2), make_neg (Pn), C)`

### Tâche 1 (facultatif)

Apportez la modification suivante à la fonction `unify`: l'en-tête de la fonction

```python
def unify(f1, f2):
    ...
    subst = {}
```

doit être modifié pour recevoir un troisième paramètre, une substitution de départ.

```python
def unify(f1, f2, subst=None):
    if not subst:
        subst = {}
```
**Rappel:** Après avoir modifié `unify` il faut télécharger `Lab08` à nouveau comme un fichier Python et redémarrer le Kernel dans ce fichier.

In [5]:
# Helpers

# Either export the function from the other practical 
# session or copy / paste them here (reccommended), in the helpers block.

# Example: (you must export it as python file)
# from Lab08_IA import make_var, make_const, make_atom, make_or, make_neg, \
#                 is_variable, is_constant, is_atom, is_function_call, \
#                 print_formula, get_args, get_head, get_name, get_value ,\
#                 unify, substitute


from copy import deepcopy
from functools import reduce

# In this cell there are several functions used internally

dummy = make_atom("P")
[and_name, or_name, neg_name] = [get_head(s) for s in [make_and(dummy, dummy), make_or(dummy, dummy), make_neg(dummy)]]
def pFail(message, f):
    print(message + " <" + str(f) + ">")
    return False
def check_term(T):
    if is_constant(T):
        return (get_value(T) is not None) or pFail("The value of the constant is None", T)
    if is_variable(T):
        return (get_name(T) is not None) or pFail("The name of the variable is None", T)
    if is_function_call(T):
        return not [t for t in get_args(T) if not check_term(t)] and \
            (get_head(T) is not None or pFail("Function is not callable", T))
    return pFail("Term is not one of constant, variable or function call", T)
def check_atom(A):
    if is_atom(A):
        return not [t for t in get_args(A) if not check_term(t)] and \
            (get_head(A) is not None or pFail("Predicate name is None", A))
    return pFail("Is not an atom", A)
def check_sentence(S):
    if is_atom(S):
        return check_atom(S)
    if is_sentence(S):
        if get_head(S) in [and_name, or_name]:
            return (len(get_args(S)) >= 2 or pFail("Sentence has too few operands", S)) \
                and not [s for s in get_args(S) if not check_sentence(s)]
        if get_head(S) == neg_name:
            return (len(get_args(S)) == 1 or pFail("Negative sentence has not just 1 operand", S)) \
                and check_sentence(get_args(S)[0])
    return pFail("Not sentence or unknown type", S)

def add_statement(kb, conclusion, *hypotheses):
    s = conclusion if not hypotheses else make_or(*([make_neg(s) for s in hypotheses] + [conclusion]))
    if check_sentence(s):
        kb.append(s)
        print("OK: Added statement " + print_formula(s, True))
        return True
    print("-- FAILED CHECK: Sentence does not check out <"+print_formula(s, True)+"><" + str(s) + ">")
    return False

var_no = 0;

def assign_next_var_name():
    global var_no
    var_no += 1
    return "v" + str(var_no)

def gather_vars(S):
    return [get_name(S)] if is_variable(S) else \
        [] if not has_args(S) else reduce(lambda res, a: res + gather_vars(a), get_args(S), [])

def make_unique_var_names(KB):
    global var_no
    var_no = 0
    return [substitute(S, {var: make_var(assign_next_var_name()) for var in gather_vars(S)}) for S in KB]           
            
def print_KB(KB):
    print("KB now:")
    for s in KB:
        print("\t\t\t" + print_formula(s, True))

ModuleNotFoundError: No module named 'Lab08_IA'

## Base de connaissances

Complétez la représentation de la base de connaissances ci-dessous en sachant qu'elle correspond aux énoncés suivants:

[TODO 2.1:] *If it rained two days in a row, the third day will be sunny.* 

[TODO 2.2:] *If it was sunny three days in a row, it will rain on the fourth day.* 

[TODO 2.3:] *A student always goes to the mountains if it is sunny on a weekend day. Those who go to the mountains and practise a winter sport will have activities related to that winter sport.*

*Mary and Kevin are students. Mary plays volleyball and skiing, and Kevin skis and sledging. Volleyball is a summer sport, while skiing and sledging are winter sports. It's raining on Friday; Monday, Tuesday and Wednesday is sunny.*

### Tâche 2 :

Remplissez les clauses définies pour les 3 premières phrases ci-dessous.

In [ ]:
def get_sports_kb():
    sports_kb = []
    # Predicate 'Consecutive'
    add_statement(sports_kb, make_atom('Consecutive', make_const('Monday'), make_const('Tuesday')))
    add_statement(sports_kb, make_atom('Consecutive', make_const('Tuesday'), make_const('Wednesday')))
    add_statement(sports_kb, make_atom('Consecutive', make_const('Wednesday'), make_const('Thursday')))
    add_statement(sports_kb, make_atom('Consecutive', make_const('Thursday'), make_const('Friday')))
    add_statement(sports_kb, make_atom('Consecutive', make_const('Friday'), make_const('Saturday')))
    add_statement(sports_kb, make_atom('Consecutive', make_const('Saturday'), make_const('Sunday')))
    # Predicate 'Weekend'
    add_statement(sports_kb, make_atom('Weekend', make_const('Saturday')))
    add_statement(sports_kb, make_atom('Weekend', make_const('Sunday')))
    # Predicate 'Rain'
    add_statement(sports_kb, make_atom('Rain', make_const('Friday')))
    # TODO 2.1: If it rained two days in a row, the third day will be sunny.
    #
    # Predicate 'Sunny'
    add_statement(sports_kb, make_atom('Sunny', make_const('Monday')))
    add_statement(sports_kb, make_atom('Sunny', make_const('Tuesday')))
    add_statement(sports_kb, make_atom('Sunny', make_const('Wednesday')))
    # TODO 2.2: If it was sunny three days in a row, it will rain on the fourth day.
    #
    # Predicate 'Student'
    add_statement(sports_kb, make_atom('Student', make_const('Mary')))
    add_statement(sports_kb, make_atom('Student', make_const('Kevin')))
    # GoesToMountain (who, when)
    # TODO 2.3: A student always goes to the mountains if it is sunny on a weekend day.
    #
    # Predicate 'SummerSport'
    add_statement(sports_kb, make_atom('SummerSport', make_const('Volleyball')))
    # Predicate 'WinterSport'
    add_statement(sports_kb, make_atom('WinterSport', make_const('Skiing')))
    add_statement(sports_kb, make_atom('WinterSport', make_const('Sledgging')))
    # Predicate 'PractiseSport'
    add_statement(sports_kb, make_atom('PractiseSport', make_const('Kevin'), make_const('Skiing')))
    add_statement(sports_kb, make_atom('PractiseSport', make_const('Kevin'), make_const('Sledgging')))
    add_statement(sports_kb, make_atom('PractiseSport', make_const('Mary'), make_const('Skiing')))
    add_statement(sports_kb, make_atom('PractiseSport', make_const('Mary'), make_const('Volleyball')))
    # Predicate 'Activity'
    add_statement(sports_kb, make_atom('Activity', make_var('who'), make_var('what'), make_var('when')),
                  make_atom('GoesToMountain', make_var('who'), make_var('when')),
                  make_atom('PractiseSport', make_var('who'), make_var('what'))
                 )
    make_unique_var_names(sports_kb)
    return sports_kb


print("This is how the knowledge base is presented:")
skb = get_sports_kb()
print_KB(skb)
print("==================== \n Inside the KB we can notice:")
print("" + "".join([(str(s) + "\n") for s in skb]))

This is how the knowledge base is presented:
OK: Added statement Consecutive(Monday, Tuesday)
OK: Added statement Consecutive(Tuesday, Wednesday)
OK: Added statement Consecutive(Wednesday, Thursday)
OK: Added statement Consecutive(Thursday, Friday)
OK: Added statement Consecutive(Friday, Saturday)
OK: Added statement Consecutive(Saturday, Sunday)
OK: Added statement Weekend(Saturday)
OK: Added statement Weekend(Sunday)
OK: Added statement Rain(Friday)
OK: Added statement Beautiful(Monday)
OK: Added statement Beautiful(Tuesday)
OK: Added statement Beautiful(Wednesday)
OK: Added statement Student(Mary)
OK: Added statement Student(Kevin)
OK: Added statement SummerSport(Volleyball)
OK: Added statement WinterSport(Skiing)
OK: Added statement WinterSport(Sledgging)
OK: Added statement PractiseSport(Kevin, Skiing)
OK: Added statement PractiseSport(Kevin, Sledgging)
OK: Added statement PractiseSport(Mary, Skiing)
OK: Added statement PractiseSport(Mary, Volleyball)
OK: Added statement (V ~GoesT

## Fonctions auxiliaires

### Tâche 3

Implémentez les fonctions `get_premises`, `get_conclusion`, `is_fact` et `is_rule`. Toutes ces fonctions reçoivent une `clause définie` (dans la base de connaissances donnée, il peut s'agir d'un seul atome ou d'une disjonction de littéraux) et renvoyer ce que leur nom spécifie.

In [ ]:
def get_premises(formula):
    # TODO
    return []

def get_conclusion(formula):
    # TODO
    return None

def is_fact(formula):
    # TODO
    return False

def is_rule(formula):
    # TODO
    return False

# Test!
# formula: P(x) ^ Q(x) -> R(x)
f = make_or(make_neg(make_atom("P", make_var("x"))), make_neg(make_atom("Q", make_var("x"))), make_atom("R", make_var("x")))
print("".join([(print_formula(p, True) + " ; ") for p in get_premises(f)])[:-3]) # Should be P(?x) ; Q(?x)
print_formula(get_conclusion(f)) # Should be R(?x)
print(is_rule(f)) # must be True
print(is_fact(get_conclusion(f))) # must be True


???
False
False


**_Résultat attendu:_**

```
P(?x) ; Q(?x)
R(?x)
True
True
```

In [ ]:
def equal_terms(t1, t2):
    if is_constant(t1) and is_constant(t2):
        return get_value(t1) == get_value(t2)
    if is_variable(t1) and is_variable(t2):
            return get_name(t1) == get_name(t2)
    if is_function_call(t1) and is_function(t2):
        if get_head(t1) != get_head(t2):
            return all([equal_terms(get_args(t1)[i], get_args(t2)[i]) for i in range(len(get_args(t1)))])
    return False

def is_equal_to(a1, a2):
    # We check atoms with the same predicate name and the same number of arguments
    if not (is_atom(a1) and is_atom(a2) and get_head(a1) == get_head(a2) and len(get_args(a1)) == len(get_args(a2))):
        return False
    return all([equal_terms(get_args(a1)[i], get_args(a2)[i]) for i in range(len(get_args(a1)))])

## Démonstration de théorèmes par enchaînement en avant

### Tâche 4

Mettez en oeuvre la fonction `apply_rule (règle, faits)` qui reçoit une règle et un ensemble de faits et renvoie tous les faits qui peuvent être déterminés en appliquant la règle aux faits donnés.

Utilisez les fonctions `unify`, `substitute`, mais aussi `get_premises` et `get_conclusion` implémentées précédemment.

In [ ]:
from copy import deepcopy
# from __future__ import print_function

def apply_rule(rule, facts):
    # TODO
    return []

# Test!
# Rule: P(x) => Q(x)
# Facts: P(1)
for f in apply_rule( 
        make_or(make_neg(make_atom("P", make_var("x"))), make_atom("Q", make_var("x"))), \
        [make_atom("P", make_const(1))]):
    print_formula(f) # should be Q(1)
print("=====")
# Rule: P(x) ^ Q(x) => R(x)
# Facts: P(1), P(2), P(3), Q(3), Q(2)
for f in apply_rule( 
        make_or(
            make_neg(make_atom("P", make_var("x"))),
            make_neg(make_atom("Q", make_var("x"))),
            make_atom("R", make_var("x"))),
        [make_atom("P", make_const(x)) for x in [1, 2, 3]] + \
        [make_atom("Q", make_const(x)) for x in [3, 2]]):
    print_formula(f) # should be R(2) and R(3)
print("=====")
# Rule: P(x) ^ Q(y) ^ R(x, y) => T(x, y)
# Facts: P(1), P(2), P(3), Q(3), Q(2), R(3, 2)
for f in apply_rule( 
        make_or(
            make_neg(make_atom("P", make_var("x"))),
            make_neg(make_atom("Q", make_var("y"))),
            make_neg(make_atom("R", make_var("x"), make_var("y"))),
            make_atom("T", make_var("x"), make_var("y"))),
        [make_atom("P", make_const(x)) for x in [1, 2, 3]] + \
        [make_atom("Q", make_const(x)) for x in [3, 2]] + \
        [make_atom("R", make_const(3), make_const(2))]):
    print_formula(f) # should be T(3, 2)

**_Résultat attendu:_**

```
Q(1)
=====
R(2)
R(3)
=====
T(3, 2)
```

In [ ]:
def forward_chaining(kb, theorem, verbose = True):
    # We save the original database, we work with a copy
    local_kb = deepcopy(kb)
    # Two variables that describe the search status
    got_new_facts = True   # new facts were found in the last search
    is_proved = False      # the theorem has been proved
    # We check if the theorem is already proved
    for fact in filter(is_fact, local_kb):
        if unify(fact, theorem):
            if verbose: print("This already in KB: " + print_formula(fact, True))
            is_proved = True
            break
    while (not is_proved) and got_new_facts:
        got_new_facts = False
        for rule in filter(is_rule, local_kb):
            # For each rule:
            new_facts = apply_rule(rule, list(filter(is_fact, local_kb)))
            new_facts = list(filter(lambda fact: not any(list(filter(lambda orig: is_equal_to(fact, orig), local_kb))), new_facts))
            if new_facts:
                if verbose: print("Applied rule: " + print_formula(rule, True))
                got_new_facts = True
                for fact in new_facts:
                    #if verbose: print("New fact: " + print_formula(fact, True))
                    if unify(fact, theorem) != False:
                        is_proved = True
                        add_statement(local_kb, fact)
                        if verbose: print("Now in KB: " + print_formula(fact, True))
                        break
                    add_statement(local_kb, fact)
            if is_proved:
                break
    if verbose:
        if is_proved:
            print("The theorem is TRUE!")
        else:
            print("The theorem is FALSE!")
    return is_proved

In [ ]:
def test_result(result, truth, idx):
    print("Test " + str(idx) + " OK!" if result == truth else "Test FAILED!")
    print("================== ")

test_result(forward_chaining(deepcopy(skb), make_atom("Sunny", make_var("x")), True), True, 0)
test_result(forward_chaining(deepcopy(skb), make_atom("Rain", make_var("x")), True), True, 1)
test_result(forward_chaining(deepcopy(skb), make_atom("Rain", make_const("Thursday")), True), True, 2)
test_result(forward_chaining(deepcopy(skb), make_atom("Sunny", make_const("Saturday")), True), True, 3)
test_result(forward_chaining(deepcopy(skb), make_atom("Activity",
                        make_const("Kevin"), make_var("Sport"), make_const("Saturday")), True), True, 4)